In [1]:
!pip install pandas numpy scikit-learn xgboost matplotlib seaborn requests gradio


In [3]:
OPENWEATHERMAP_KEY = "79cf120f97d3fce0cf1e498017236eba"
TOMTOM_KEY = "kZz5Has4cOabIOrIcazlsYHbTlAxBTSL"

In [4]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

np.random.seed(42)

n_orders = 50000
restaurants = [f"Restaurant_{i}" for i in range(1, 101)]
cuisines = ["North Indian", "South Indian", "Chinese", "Italian", "Biryani", "Desserts"]
delivery_partners = [f"DP_{i}" for i in range(1, 301)]

data = []
start_date = datetime(2024, 1, 1)

for _ in range(n_orders):
    rest = random.choice(restaurants)
    cuisine = random.choice(cuisines)
    partner = random.choice(delivery_partners)
    order_time = start_date + timedelta(minutes=random.randint(0, 60*24*90))
    items_count = np.random.randint(1, 6)
    distance_km = round(np.random.uniform(0.5, 8.0), 2)
    weather_delay = np.random.choice([0, 5, 10, 15], p=[0.7, 0.15, 0.1, 0.05])
    traffic_delay = np.random.choice([0, 3, 7, 12], p=[0.6, 0.2, 0.15, 0.05])
    prep_time = np.random.randint(10, 35)

    base_time = prep_time + distance_km*3 + weather_delay + traffic_delay
    noise = np.random.normal(0, 3)
    actual_time = max(10, round(base_time + noise, 2))

    data.append([rest, cuisine, partner, order_time, items_count, distance_km,
                 weather_delay, traffic_delay, prep_time, actual_time])

df = pd.DataFrame(data, columns=[
    "restaurant", "cuisine", "delivery_partner", "order_time",
    "items_count", "distance_km", "weather_delay", "traffic_delay",
    "prep_time", "actual_delivery_time"
])

df.to_csv("swiggy_orders.csv", index=False)
df.head()


,restaurant,cuisine,delivery_partner,order_time,items_count,distance_km,weather_delay,traffic_delay,prep_time,actual_delivery_time
0,Restaurant_98,North Indian,DP_50,2024-02-21 01:48:00,4,7.63,5,0,16,41.14
1,Restaurant_40,Desserts,DP_26,2024-03-06 18:58:00,3,7.00,0,3,31,54.63
2,Restaurant_46,Chinese,DP_154,2024-03-14 06:30:00,5,7.77,5,0,30,62.70
3,Restaurant_53,Biryani,DP_183,2024-01-10 14:00:00,5,3.74,0,3,19,37.84
4,Restaurant_77,South Indian,DP_237,2024-02-27 07:46:00,4,7.80,0,0,28,51.83


In [5]:
import requests

def get_current_weather():
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {"q": "New Delhi,IN", "appid": OPENWEATHERMAP_KEY, "units": "metric"}
    res = requests.get(url, params=params).json()
    temp = res["main"]["temp"]
    weather_desc = res["weather"][0]["description"]
    return temp, weather_desc

def get_current_traffic(origin_lat=28.6139, origin_lon=77.2090, dest_lat=28.7041, dest_lon=77.1025):
    url = f"https://api.tomtom.com/routing/1/calculateRoute/{origin_lat},{origin_lon}:{dest_lat},{dest_lon}/json"
    params = {"key": TOMTOM_KEY, "traffic": "true"}
    res = requests.get(url, params=params).json()
    try:
        travel_time_sec = res["routes"][0]["summary"]["travelTimeInSeconds"]
        return travel_time_sec / 60
    except:
        return 5  # default delay if API fails


In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import xgboost as xgb
import pickle

df = pd.read_csv("swiggy_orders.csv", parse_dates=["order_time"])

df["hour"] = df["order_time"].dt.hour
df["day_of_week"] = df["order_time"].dt.dayofweek
df["is_weekend"] = df["day_of_week"].isin([5, 6]).astype(int)
df.drop(columns=["order_time"], inplace=True)

for col in ["restaurant", "cuisine", "delivery_partner"]:
    df[col] = LabelEncoder().fit_transform(df[col])

X = df.drop(columns=["actual_delivery_time"])
y = df["actual_delivery_time"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = xgb.XGBRegressor(n_estimators=300, learning_rate=0.1, max_depth=8, subsample=0.8, colsample_bytree=0.8)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print(f"RMSE: {rmse:.2f} mins | MAE: {mae:.2f} mins")

pickle.dump(model, open("eta_model.pkl", "wb"))


RMSE: 3.17 mins | MAE: 2.53 mins


In [13]:
# import gradio as gr

# # Load model and data
# model = pickle.load(open("eta_model.pkl", "rb"))
# df = pd.read_csv("swiggy_orders.csv")

# # Extract unique restaurant/cuisine/delivery partner lists
# restaurant_list = sorted(df["restaurant"].unique())
# cuisine_list = sorted(df["cuisine"].unique())
# dp_list = sorted(df["delivery_partner"].unique())

# # Refit label encoders so we can map names -> encoded integers
# from sklearn.preprocessing import LabelEncoder

# le_rest = LabelEncoder().fit(restaurant_list)
# le_cuisine = LabelEncoder().fit(cuisine_list)
# le_dp = LabelEncoder().fit(dp_list)

# def predict_eta(restaurant_name, cuisine_name, delivery_partner_name, items_count, distance_km, prep_time):
#     # Encode names to numeric values
#     rest_id = le_rest.transform([restaurant_name])[0]
#     cuisine_id = le_cuisine.transform([cuisine_name])[0]
#     dp_id = le_dp.transform([delivery_partner_name])[0]

#     # Get live API data
#     temp, desc = get_current_weather()
#     traffic_time = get_current_traffic()
#     weather_delay = 5 if "rain" in desc.lower() else 0

#     # Static time features for demo
#     hour = 12
#     day_of_week = 3
#     is_weekend = 0

#     # Prepare input for prediction
#     input_df = pd.DataFrame([[rest_id, cuisine_id, dp_id, items_count, distance_km,
#                               weather_delay, traffic_time, prep_time, hour, day_of_week, is_weekend]],
#                             columns=['restaurant', 'cuisine', 'delivery_partner', 'items_count',
#                                      'distance_km', 'weather_delay', 'traffic_delay', 'prep_time',
#                                      'hour', 'day_of_week', 'is_weekend'])

#     # Predict ETA
#     eta = model.predict(input_df)[0]
#     return f"ETA: {eta:.2f} minutes | Weather: {temp}°C, {desc} | Traffic Delay: {traffic_time:.2f} mins"

# # Gradio UI
# iface = gr.Interface(
#     fn=predict_eta,
#     inputs=[
#         gr.Dropdown(choices=restaurant_list, label="Select Restaurant"),
#         gr.Dropdown(choices=cuisine_list, label="Select Cuisine"),
#         gr.Dropdown(choices=dp_list, label="Select Delivery Partner"),
#         gr.Number(label="Items Count", value=2),
#         gr.Number(label="Distance (km)", value=3.0),
#         gr.Number(label="Preparation Time (mins)", value=20)
#     ],
#     outputs="text",
#     title="🚀 SwiggyETA+ - Smart Delivery Time Prediction"
# )

# iface.launch()


In [15]:
# # ===============================
# # SwiggyETA+ - Smart Delivery Time Prediction
# # ===============================

# !pip install gradio scikit-learn requests

# import gradio as gr
# import pandas as pd
# import numpy as np
# import requests
# import random
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.metrics import mean_squared_error, mean_absolute_error
# import joblib

# # ===============================
# # API Keys
# # ===============================
# OPENWEATHERMAP_KEY = "79cf120f97d3fce0cf1e498017236eba"
# TOMTOM_KEY = "kZz5Has4cOabIOrIcazlsYHbTlAxBTSL"

# # ===============================
# # Step 1: Create Synthetic Historical Data
# # ===============================
# restaurants = ["Biryani Blues", "Pizza Hut", "Haldiram's", "Burger King", "Subway"]
# cuisines = ["Biryani", "Pizza", "North Indian", "Fast Food", "Sandwiches"]
# delivery_partners = ["DP_Raj", "DP_Amit", "DP_Sunil", "DP_Meena", "DP_Anjali"]

# np.random.seed(42)
# data = []
# for _ in range(500):
#     r = random.choice(restaurants)
#     c = random.choice(cuisines)
#     dp = random.choice(delivery_partners)
#     items = random.randint(1, 6)
#     dist = round(random.uniform(1, 10), 2)
#     prep = random.randint(10, 40)
#     weather_temp = round(random.uniform(15, 40), 1)
#     traffic_delay = random.randint(0, 15)
#     delivery_time = prep + dist * 4 + traffic_delay + random.randint(-3, 3)
#     data.append([r, c, dp, items, dist, prep, weather_temp, traffic_delay, delivery_time])

# df = pd.DataFrame(data, columns=[
#     "Restaurant", "Cuisine", "DeliveryPartner", "ItemsCount",
#     "Distance_km", "PrepTime_mins", "Temp_C", "TrafficDelay_mins", "DeliveryTime_mins"
# ])

# # ===============================
# # Step 2: Train Model
# # ===============================
# X = df.drop(columns=["DeliveryTime_mins"])
# y = df["DeliveryTime_mins"]

# categorical_cols = ["Restaurant", "Cuisine", "DeliveryPartner"]
# encoder = OneHotEncoder(handle_unknown="ignore")
# X_cat = encoder.fit_transform(X[categorical_cols])
# X_num = X.drop(columns=categorical_cols).values
# X_all = np.hstack([X_cat.toarray(), X_num])

# X_train, X_test, y_train, y_test = train_test_split(X_all, y, test_size=0.2, random_state=42)
# model = RandomForestRegressor(n_estimators=100, random_state=42)
# model.fit(X_train, y_train)

# y_pred = model.predict(X_test)
# print("RMSE:", round(np.sqrt(mean_squared_error(y_test, y_pred)), 2), "mins | MAE:", round(mean_absolute_error(y_test, y_pred), 2), "mins")

# # Save model and encoder
# joblib.dump(model, "model.pkl")
# joblib.dump(encoder, "encoder.pkl")

# # ===============================
# # Step 3: API Functions
# # ===============================
# import datetime

# def get_current_weather():
#     try:
#         url = f"https://api.openweathermap.org/data/2.5/weather?q=Delhi&appid={OPENWEATHERMAP_KEY}&units=metric"
#         r = requests.get(url, timeout=5)
#         r.raise_for_status()  # Raise HTTP error if any
#         data = r.json()
#         temp = data["main"]["temp"]
#         desc = data["weather"][0]["description"]
#         print(f"[{datetime.datetime.now()}] Weather API success: {temp}°C, {desc}")
#         return round(temp, 1), desc
#     except Exception as e:
#         print(f"[{datetime.datetime.now()}] Weather API error: {e}")
#         return 30.0, "clear sky (default)"

# def get_current_traffic():
#     try:
#         start_lat, start_lon = 28.6315, 77.2167  # Connaught Place
#         end_lat, end_lon = 28.6129, 77.2295      # India Gate
#         url = f"https://api.tomtom.com/routing/1/calculateRoute/{start_lat},{start_lon}:{end_lat},{end_lon}/json?key={TOMTOM_KEY}&traffic=true"
#         r = requests.get(url, timeout=5)
#         r.raise_for_status()
#         data = r.json()
#         delay_sec = data["routes"][0]["summary"]["trafficDelayInSeconds"]
#         print(f"[{datetime.datetime.now()}] Traffic API success: {round(delay_sec/60, 1)} mins delay")
#         return round(delay_sec / 60, 1)
#     except Exception as e:
#         print(f"[{datetime.datetime.now()}] Traffic API error: {e}")
#         return random.randint(2, 10)


# # ===============================
# # Step 4: Prediction Function
# # ===============================
# def predict_eta(restaurant, cuisine, partner, items, distance, prep_time):
#     temp, weather_desc = get_current_weather()
#     traffic_delay = get_current_traffic()

#     model = joblib.load("model.pkl")
#     encoder = joblib.load("encoder.pkl")

#     row = pd.DataFrame([[restaurant, cuisine, partner, items, distance, prep_time, temp, traffic_delay]],
#                        columns=["Restaurant", "Cuisine", "DeliveryPartner", "ItemsCount", "Distance_km", "PrepTime_mins", "Temp_C", "TrafficDelay_mins"])

#     X_cat = encoder.transform(row[categorical_cols])
#     X_num = row.drop(columns=categorical_cols).values
#     X_all = np.hstack([X_cat.toarray(), X_num])

#     pred_time = model.predict(X_all)[0]
#     return f"📦 ETA: {round(pred_time, 1)} mins\n🌤 Weather: {temp}°C, {weather_desc}\n🚦 Traffic Delay: {traffic_delay} mins"

# # ===============================
# # Step 5: Gradio Interface
# # ===============================
# with gr.Blocks() as demo:
#     gr.Markdown("## 🚀 SwiggyETA+ - Smart Delivery Time Prediction")

#     with gr.Row():
#         with gr.Column():
#             rest_dd = gr.Dropdown(restaurants, label="Select Restaurant")
#             cuisine_dd = gr.Dropdown(cuisines, label="Select Cuisine")
#             partner_dd = gr.Dropdown(delivery_partners, label="Select Delivery Partner")
#             items_in = gr.Number(label="Items Count", value=2)
#             dist_in = gr.Number(label="Distance (km)", value=4)
#             prep_in = gr.Number(label="Preparation Time (mins)", value=20)
#             submit_btn = gr.Button("Submit")

#         with gr.Column():
#             output_box = gr.Textbox(label="Prediction Output")

#     submit_btn.click(
#         fn=predict_eta,
#         inputs=[rest_dd, cuisine_dd, partner_dd, items_in, dist_in, prep_in],
#         outputs=output_box
#     )

# demo.launch(share=True)


In [ ]:
# =========================
# 📦 Swiggy-Style ETA Predictor with Live Weather & Traffic
# =========================

import pandas as pd
import numpy as np
import random
import requests
import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import gradio as gr

# =========================
# 🔑 API Keys (replace with yours)
# =========================
OPENWEATHERMAP_KEY = "79cf120f97d3fce0cf1e498017236eba"
TOMTOM_KEY = "kZz5Has4cOabIOrIcazlsYHbTlAxBTSL"

# =========================
# 📊 Step 1: Synthetic Dataset
# =========================
restaurants = ["Biryani Blues", "Haldiram's", "Burger Singh", "Domino's Pizza", "Behrouz Biryani"]
cuisines = ["North Indian", "South Indian", "Chinese", "Pizza", "Biryani"]

np.random.seed(42)
data = {
    "restaurant": np.random.choice(restaurants, 200),
    "cuisine": np.random.choice(cuisines, 200),
    "order_size": np.random.randint(1, 6, 200),
    "distance_km": np.random.uniform(0.5, 10, 200).round(2),
    "prep_time": np.random.randint(5, 25, 200),  # minutes
    "temp_c": np.random.uniform(20, 35, 200).round(1),
    "traffic_delay_mins": np.random.randint(0, 15, 200),
}
data["delivery_time_mins"] = (
    data["prep_time"] + data["distance_km"] * 3 + data["traffic_delay_mins"] + np.random.uniform(-2, 2, 200)
).round(1)

df = pd.DataFrame(data)

# =========================
# 🧠 Step 2: Train Model
# =========================
X = df.drop(columns=["delivery_time_mins"])
y = df["delivery_time_mins"]

X = pd.get_dummies(X, columns=["restaurant", "cuisine"], drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

preds = model.predict(X_test)

# ✅ RMSE fix for older scikit-learn
rmse = mean_squared_error(y_test, preds) ** 0.5  # sqrt of MSE
mae = mean_absolute_error(y_test, preds)
print(f"✅ Model trained | RMSE: {rmse:.2f} mins | MAE: {mae:.2f} mins")

# =========================
# 🌤 Step 3: Live Weather API (with Logging)
# =========================
def get_current_weather():
    try:
        url = f"https://api.openweathermap.org/data/2.5/weather?q=Delhi&appid={OPENWEATHERMAP_KEY}&units=metric"
        r = requests.get(url, timeout=5)
        r.raise_for_status()
        data = r.json()
        temp = data["main"]["temp"]
        desc = data["weather"][0]["description"]
        print(f"[{datetime.datetime.now()}] Weather API success: {temp}°C, {desc}")
        return round(temp, 1), desc
    except Exception as e:
        print(f"[{datetime.datetime.now()}] Weather API error: {e}")
        return 30.0, "clear sky (default)"

# =========================
# 🚦 Step 4: Live Traffic API (with Logging)
# =========================
def get_current_traffic():
    try:
        start_lat, start_lon = 28.6315, 77.2167  # Connaught Place
        end_lat, end_lon = 28.6129, 77.2295      # India Gate
        url = f"https://api.tomtom.com/routing/1/calculateRoute/{start_lat},{start_lon}:{end_lat},{end_lon}/json?key={TOMTOM_KEY}&traffic=true"
        r = requests.get(url, timeout=5)
        r.raise_for_status()
        data = r.json()
        delay_sec = data["routes"][0]["summary"]["trafficDelayInSeconds"]
        print(f"[{datetime.datetime.now()}] Traffic API success: {round(delay_sec/60, 1)} mins delay")
        return round(delay_sec / 60, 1)
    except Exception as e:
        print(f"[{datetime.datetime.now()}] Traffic API error: {e}")
        return random.randint(2, 10)

# =========================
# 🚚 Step 5: Prediction Function
# =========================
def predict_eta(restaurant, cuisine, order_size, distance_km):
    temp_c, weather_desc = get_current_weather()
    traffic_delay = get_current_traffic()

    input_df = pd.DataFrame([{
        "restaurant": restaurant,
        "cuisine": cuisine,
        "order_size": order_size,
        "distance_km": distance_km,
        "prep_time": random.randint(5, 20),
        "temp_c": temp_c,
        "traffic_delay_mins": traffic_delay
    }])

    input_df = pd.get_dummies(input_df, columns=["restaurant", "cuisine"], drop_first=True)

    for col in X.columns:
        if col not in input_df.columns:
            input_df[col] = 0
    input_df = input_df[X.columns]

    eta = model.predict(input_df)[0]
    return f"📦 ETA: {eta:.1f} mins\n🌤 Weather: {temp_c}°C, {weather_desc}\n🚦 Traffic Delay: {traffic_delay} mins"

# =========================
# 🖥 Step 6: Gradio Interface
# =========================
restaurant_list = restaurants
cuisine_list = cuisines

iface = gr.Interface(
    fn=predict_eta,
    inputs=[
        gr.Dropdown(restaurant_list, label="Restaurant"),
        gr.Dropdown(cuisine_list, label="Cuisine"),
        gr.Slider(1, 5, step=1, label="Order Size"),
        gr.Slider(0.5, 10, step=0.1, label="Distance (km)"),
    ],
    outputs="text",
    title="Swiggy-style Real-Time Delivery ETA Predictor",
    description="Predicts delivery ETA using ML + live Delhi weather & traffic data.",
)

iface.launch(debug=True, share=True)


✅ Model trained | RMSE: 2.97 mins | MAE: 2.25 mins
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c305dd6c6c67a71c15.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
